In [1]:
!pip install langchain
!pip install langchain-openai
!pip install python-dotenv
!pip install pypdf
!pip install PyPDF2
!pip install chromadb
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 656.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [4]:
%reload_ext autoreload
%autoreload 2


In [5]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

False

In [14]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.1 MB/s eta 0:00:00


In [20]:

from langchain.llms import OpenAI

llm = OpenAI(openai_api_key="your_api_key")

In [21]:
llm.invoke("What is a legally binding contract and its termination period?")

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************6WCu. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

**Working with LangChainExpressionLanguage(LCEL) to create a chain**

In [2]:
from langchain_core.prompts import ChatPromptTemplate

# add output parser to the chain, specifically string parser for the output
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert in reviewing a contract between Cloud Investments and Mr Jack Robinson"),
    ("user", "{input}")
])

In [3]:
chain = prompt | llm | output_parser

NameError: name 'llm' is not defined


**Asking our LLM a question regarding the contract**

Since it doesn't actually know about the contract between the 2 parties, it won't give us the direct/correct answer we want


In [ ]:
chain.invoke({"input": "Who owns the IP"})

**Working on the retriever**

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../data/Raptor Contract.pdf")
pages = loader.load()


In [ ]:
len(pages)

In [ ]:
# Splitting the documents into chunks
from langchain.text_splitter  import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000, chunk_overlap=200, length_function=len
)
document_chunks = text_splitter.split_documents(pages)
len(document_chunks)


In [ ]:
# storing the chunks in a vector store
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

# save to disk
db = Chroma.from_documents(pages, embeddings, persist_directory="../data/chroma_db")

In [ ]:
# load from disk
vector_store = Chroma(persist_directory="../data/chroma_db", embedding_function=embeddings)

query = 'What does Closing Debt Amount mean?'
docs_retrieved = vector_store.similarity_search(query)
print(docs_retrieved[0].page_content)


**Create a RAG Chain**

In [ ]:
# create chain for documents
from langchain.chains.combine_documents import create_stuff_documents_chain

template = """"Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}
"""
prompt = ChatPromptTemplate.from_template(template)
document_chain = create_stuff_documents_chain(llm, prompt)



In [ ]:
# create retrieval chain

from langchain.chains import create_retrieval_chain

retriever = vector_store.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({
    "input": "What does Closing Debt Amount mean?"
})
response
